In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [20]:
df = pd.read_csv('./NuclearCytosolLncRNAs_ALL_4mer_stride1_tokens.csv.count.csv',index_col=0)

In [5]:
df.head()

,class,aaaa,tttt,agaa,cctg,ccag,ctgg,aaat,gaaa,cagg,...,acga,ttcg,tcgt,cgac,gtcg,tcga,atcg,cgat,tacg,cgta
name,,,,,,,,,,,,,,,,,,,,,
ENST00000589451.1,Nuclear,0,0,3,8,10,12,1,2,7,...,0,0,0,0,0,0,0,0,0,0
ENST00000490351.1,Nuclear,8,5,9,1,4,2,4,5,2,...,2,0,0,1,0,1,0,0,0,0
ENST00000527727.1,Nuclear,8,4,4,3,4,6,4,6,3,...,0,0,0,0,0,0,0,0,0,0
ENST00000612724.1,Nuclear,6,4,4,2,4,4,2,2,1,...,0,0,0,0,2,0,0,0,0,0
ENST00000634547.1,Nuclear,5,6,3,12,12,14,5,7,10,...,1,1,1,0,2,0,0,2,1,1


In [6]:
df.shape

(8662, 257)

In [21]:
# convert the classes to 0 and 1
df['class'] = pd.factorize(df['class'])[0]

In [22]:
# Strtify sample to train , test and validation data
# Validation data is important for neural network backpropagation
X=df.drop(columns=['class'])
y=df.loc[:,'class']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=15)
X_train, X_valid, y_train, y_valid = train_test_split(X_train,y_train,test_size=0.15, random_state=15)

In [23]:
## SparsePCA is this running a Principal Components Analysis for feature selection?

In [24]:
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.decomposition import SparsePCA,PCA
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.feature_selection import VarianceThreshold

In [25]:
# A pipeline is a list of preprocessing steps in one command
# first we will remove high correlated features
# then we will apply scaling as neural network needs values frm 0 to 1
pipeline = Pipeline([('corr-removal',VarianceThreshold(0.8)),
                     #('selector',SelectKBest(chi2,100)),
                     ('scaler',MinMaxScaler())])

In [26]:
# Applying the pipeline
x_train = pipeline.fit_transform(X_train,y_train)
# notice that the pipeline learn the best parameters from the tarin data only to apply to test ana valiadation data
x_test = pipeline.transform(X_test)
x_valid = pipeline.transform(X_valid)

C:\Users\amorrow\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [30]:
import sys
!{sys.executable} -m pip install tensorflow

  Running setup.py bdist_wheel for termcolor: started
  Running setup.py bdist_wheel for termcolor: finished with status 'error'
  Complete output from command C:\Users\amorrow\AppData\Local\Continuum\anaconda3\python.exe -u -c "import setuptools, tokenize;__file__='C:\\Users\\amorrow\\AppData\\Local\\Temp\\pip-install-jbzobkl8\\termcolor\\setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d C:\Users\amorrow\AppData\Local\Temp\pip-wheel-g76wuclr --python-tag cp36:
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib
  copying termcolor.py -> build\lib
  installing to build\bdist.win-amd64\wheel
  running install
  running install_lib
  creating build\bdist.win-amd64
  creating build\bdist.win-amd64\wheel
  copying build\lib\termcolor.py -> build\bdist.win-amd64\wheel\.
  running install_egg_info
  running egg_info
  creating termcolor.egg-info
 

  Failed building wheel for termcolor
distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 19.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [28]:
python -m pip install --upgrade pip' command.

SyntaxError: invalid syntax (<ipython-input-28-8251ee19c170>, line 1)

In [31]:
import keras
from keras.layers import Dense, Dropout, Conv1D
from keras.models import Model, Sequential
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Concatenate, LeakyReLU, concatenate, MaxPool1D,GlobalMaxPool1D,add
from keras.layers import Dense, Embedding, Input, Masking, Dropout, MaxPooling1D,Lambda, BatchNormalization
from keras.layers import LSTM, TimeDistributed, AveragePooling1D, Flatten,Activation,ZeroPadding1D, UpSampling1D
from keras.optimizers import Adam, rmsprop
from keras.callbacks import ReduceLROnPlateau, EarlyStopping,ModelCheckpoint, CSVLogger
from keras.layers import Conv1D, GlobalMaxPooling1D, ConvLSTM2D, Bidirectional,RepeatVector
from keras import regularizers
from keras.utils import plot_model, to_categorical

Using TensorFlow backend.


In [32]:
# Buid the MLP neural network 
def build_mlp_classifier():
    #the input must have the number of features
    inp1 = Input(shape=(x_train.shape[1],),dtype='float')
    main = Dense(64)(inp1)
    # Dropouts are important t prevent the overfitting
    main = Dropout(0.5)(main)
    # Batch normalization allow faster convergence
    main = BatchNormalization()(main)
    main = Dense(64)(main)  
    main = Dropout(0.5)(main)
    main = BatchNormalization()(main)
    main = Dense(32)(main)  
    #main = Dropout(0.5)(main)
    #main = Dense(256)(main)  
    #main = Dropout(0.5)(main)
    # Simoid function is a must in binary classification
    out = Dense(2,activation='sigmoid')(main)
    model = Model(inputs=[inp1], outputs=[out])
    # Slower leraning rate is important in small dataset < 0.001
    optimizer = Adam(lr=0.0001)
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,metrics=['accuracy'])
    return model

In [33]:
# Converting the output to one hot encoding
y_train_ = to_categorical(y_train,num_classes=2)
y_test_ = to_categorical(y_test,num_classes=2)
y_valid_ = to_categorical(y_valid,num_classes=2)

In [34]:
y_train_[:5]

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.]], dtype=float32)

In [53]:
# MLP model
batch_size = 32
model = build_mlp_classifier()
# check point to save the model at best epoch
checkpoint = ModelCheckpoint(filepath='MLP.hdfs', save_weights_only=True,
                             monitor='val_loss',save_best_only=True)
#early top to stop the model if don't converge in 30 epochs
EarlyStop = EarlyStopping(monitor='val_loss', min_delta=0, patience=30,
                          verbose=0, mode='auto')
# RL to decrease the learning rate if the model didn't converge in 10 epochs
RL = ReduceLROnPlateau(min_lr=0.00001,factor=0.3,patience=10)
model.fit(x_train,y_train_,callbacks=[checkpoint,RL,EarlyStop],
          validation_data=[x_valid, y_valid_],
          epochs=1000,
          batch_size=batch_size, 
          verbose=2)
# loading the bestw weights
model.load_weights('MLP.hdfs')

# Calcualting the accuracy
acc = model.evaluate(x_test,y_test_)[1]
print('MLP Model Accuracy : '+ str(acc))

Train on 1359 samples, validate on 240 samples
Epoch 1/1000
 - 2s - loss: 0.8457 - acc: 0.5184 - val_loss: 0.7023 - val_acc: 0.5229
Epoch 2/1000
 - 0s - loss: 0.8114 - acc: 0.5453 - val_loss: 0.6926 - val_acc: 0.5333
Epoch 3/1000
 - 0s - loss: 0.7814 - acc: 0.5493 - val_loss: 0.6829 - val_acc: 0.5563
Epoch 4/1000
 - 0s - loss: 0.7606 - acc: 0.5475 - val_loss: 0.6782 - val_acc: 0.5750
Epoch 5/1000
 - 0s - loss: 0.7297 - acc: 0.5762 - val_loss: 0.6776 - val_acc: 0.5729
Epoch 6/1000
 - 0s - loss: 0.7271 - acc: 0.5633 - val_loss: 0.6705 - val_acc: 0.5771
Epoch 7/1000
 - 0s - loss: 0.7357 - acc: 0.5740 - val_loss: 0.6681 - val_acc: 0.5771
Epoch 8/1000
 - 0s - loss: 0.7308 - acc: 0.5820 - val_loss: 0.6653 - val_acc: 0.5750
Epoch 9/1000
 - 0s - loss: 0.7345 - acc: 0.5636 - val_loss: 0.6605 - val_acc: 0.5771
Epoch 10/1000
 - 0s - loss: 0.6980 - acc: 0.6026 - val_loss: 0.6595 - val_acc: 0.5875
Epoch 11/1000
 - 0s - loss: 0.7057 - acc: 0.5872 - val_loss: 0.6582 - val_acc: 0.5958
Epoch 12/1000
 -

Epoch 96/1000
 - 0s - loss: 0.5999 - acc: 0.6685 - val_loss: 0.6321 - val_acc: 0.6521
Epoch 97/1000
 - 0s - loss: 0.5956 - acc: 0.6784 - val_loss: 0.6321 - val_acc: 0.6583
Epoch 98/1000
 - 0s - loss: 0.5992 - acc: 0.6979 - val_loss: 0.6322 - val_acc: 0.6521
Epoch 99/1000
 - 0s - loss: 0.5998 - acc: 0.6770 - val_loss: 0.6322 - val_acc: 0.6583
Epoch 100/1000
 - 0s - loss: 0.5973 - acc: 0.6854 - val_loss: 0.6324 - val_acc: 0.6458
Epoch 101/1000
 - 0s - loss: 0.5944 - acc: 0.6840 - val_loss: 0.6321 - val_acc: 0.6583
Epoch 102/1000
 - 0s - loss: 0.5988 - acc: 0.6818 - val_loss: 0.6323 - val_acc: 0.6500
Epoch 103/1000
 - 0s - loss: 0.5982 - acc: 0.6707 - val_loss: 0.6320 - val_acc: 0.6500
Epoch 104/1000
 - 0s - loss: 0.5995 - acc: 0.6685 - val_loss: 0.6318 - val_acc: 0.6479
400/400 [==============================] - 0s 53us/step
MLP Model Accuracy : 0.62


In [54]:
# Testing some simple ML

In [39]:
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [40]:
clf = RandomForestClassifier()
clf.fit(x_train,y_train)
pred = clf.predict(x_test)
accuracy_score(pred,y_test)

0.6410848240046163

In [41]:
clf = LogisticRegression()
clf.fit(x_train,y_train)
pred = clf.predict(x_test)
accuracy_score(pred,y_test)

0.6739757645701097

### tpot apply genetic algorithm for hyper parameter optimization

In [38]:
# it takes alot of time
# you will need to install tpot first pip install tpot

In [268]:
from tpot import TPOTClassifier
tpot = TPOTClassifier(verbosity=3,cv=10,#config_dict=tpot_config,#max_time_mins=10,
 scoring='accuracy', 
 random_state=25, 
 periodic_checkpoint_folder='tpot_results.txt', 
 n_jobs=-1, 
 generations=40, 
 population_size=10,
 early_stop = 5)
tpot.fit(x_train,y_train)

/home/biohacker2/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


30 operators have been imported by TPOT.


Created new folder to save periodic pipeline: tpot_results.txt
Saving best periodic pipeline to tpot_results.txt/pipeline_2019.01.23_20-08-24.py
_pre_test decorator: _random_mutation_operator: num_test=0 Input X must be non-negative
Generation 1 - Current Pareto front scores:
-1	0.6474036703083503	XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.001, XGBClassifier__max_depth=7, XGBClassifier__min_child_weight=7, XGBClassifier__n_estimators=100, XGBClassifier__nthread=1, XGBClassifier__subsample=0.9500000000000001)
-2	0.6556923174784919	KNeighborsClassifier(XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.5, XGBClassifier__max_depth=1, XGBClassifier__min_child_weight=9, XGBClassifier__n_estimators=100, XGBClassifier__nthread=1, XGBClassifier__subsample=0.9500000000000001), KNeighborsClassifier__n_neighbors=62, KNeighborsClassifier__p=2, KNeighborsClassifier__weights=distance)

Saving best periodic pipeline to tpot_results.txt/pipeline_2019.01.23_20-09-18.py
Pipeline 

Skipped pipeline #138 due to time out. Continuing to the next pipeline.
Generation 13 - Current Pareto front scores:
-1	0.6732483739643464	XGBClassifier(CombineDFs(input_matrix, input_matrix), XGBClassifier__learning_rate=0.01, XGBClassifier__max_depth=7, XGBClassifier__min_child_weight=3, XGBClassifier__n_estimators=100, XGBClassifier__nthread=1, XGBClassifier__subsample=0.7000000000000001)

Saving best periodic pipeline to tpot_results.txt/pipeline_2019.01.23_20-52-18.py
_pre_test decorator: _random_mutation_operator: num_test=0 Input X must be non-negative
Generation 14 - Current Pareto front scores:
-1	0.6732483739643464	XGBClassifier(CombineDFs(input_matrix, input_matrix), XGBClassifier__learning_rate=0.01, XGBClassifier__max_depth=7, XGBClassifier__min_child_weight=3, XGBClassifier__n_estimators=100, XGBClassifier__nthread=1, XGBClassifier__subsample=0.7000000000000001)

Periodic pipeline was not saved, probably saved before...
_pre_test decorator: _random_mutation_operator: num_

Generation 22 - Current Pareto front scores:
-1	0.6732483739643464	XGBClassifier(CombineDFs(input_matrix, input_matrix), XGBClassifier__learning_rate=0.01, XGBClassifier__max_depth=7, XGBClassifier__min_child_weight=3, XGBClassifier__n_estimators=100, XGBClassifier__nthread=1, XGBClassifier__subsample=0.7000000000000001)
-2	0.6768871952674014	XGBClassifier(OneHotEncoder(input_matrix, OneHotEncoder__minimum_fraction=0.15, OneHotEncoder__sparse=False, OneHotEncoder__threshold=10), XGBClassifier__learning_rate=0.01, XGBClassifier__max_depth=7, XGBClassifier__min_child_weight=3, XGBClassifier__n_estimators=100, XGBClassifier__nthread=1, XGBClassifier__subsample=0.7000000000000001)
-4	0.6769684175373312	XGBClassifier(CombineDFs(RFE(StandardScaler(input_matrix), RFE__ExtraTreesClassifier__criterion=gini, RFE__ExtraTreesClassifier__max_features=0.3, RFE__ExtraTreesClassifier__n_estimators=100, RFE__step=0.6000000000000001), VarianceThreshold(input_matrix, VarianceThreshold__threshold=0.001)),

_pre_test decorator: _random_mutation_operator: num_test=0 Found array with 0 feature(s) (shape=(50, 0)) while a minimum of 1 is required.
Generation 30 - Current Pareto front scores:
-1	0.6769684175373312	ExtraTreesClassifier(input_matrix, ExtraTreesClassifier__bootstrap=False, ExtraTreesClassifier__criterion=entropy, ExtraTreesClassifier__max_features=0.3, ExtraTreesClassifier__min_samples_leaf=16, ExtraTreesClassifier__min_samples_split=19, ExtraTreesClassifier__n_estimators=100)
-2	0.6849813542610881	RandomForestClassifier(CombineDFs(input_matrix, ZeroCount(input_matrix)), RandomForestClassifier__bootstrap=False, RandomForestClassifier__criterion=gini, RandomForestClassifier__max_features=0.2, RandomForestClassifier__min_samples_leaf=1, RandomForestClassifier__min_samples_split=11, RandomForestClassifier__n_estimators=100)

Periodic pipeline was not saved, probably saved before...
_pre_test decorator: _random_mutation_operator: num_test=0 Expected n_neighbors <= n_samples,  but n_s

TPOTClassifier(config_dict=None, crossover_rate=0.1, cv=10,
        disable_update_check=False, early_stop=5, generations=40,
        max_eval_time_mins=5, max_time_mins=None, memory=None,
        mutation_rate=0.9, n_jobs=-1, offspring_size=None,
        periodic_checkpoint_folder='tpot_results.txt', population_size=10,
        random_state=25, scoring='accuracy', subsample=1.0, use_dask=False,
        verbosity=3, warm_start=False)

In [42]:
y_hat_train = tpot.predict(x_train)
train_acc = accuracy_score(y_train,y_hat_train)
print("Train set Accuracy for ensemble: " + str(train_acc))

y_pred = tpot.predict(x_test)
test_acc = accuracy_score(y_test,y_pred) 
print("Test Set Accuracy for ensemble: " + str(test_acc))

NameError: name 'tpot' is not defined